In [1]:
import sys
import os
import argparse
import random

SCRIPT_DIR = os.getcwd()
PROJECT_ROOT = os.path.dirname(SCRIPT_DIR)
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# Third-party
import numpy as np
import torch
import tensorflow as tf
from datasets import load_dataset

# Communication
from basestation import User, EdgeServer
from user_association import uncertainty_aware_offloading
from utils import (
    estimate_workload,
    generate_rayleigh_coeffs,
    bit_size_text,
)
from config import (
    BANDWIDTH,
    TRANSMIT_POWER,
    NOISE_POWER,
    LOCAL_COMPUTE_CAP,
    EDGE_COMPUTE_CAP,
    MAX_COMPUTE_PER_USER,
    SLM,
    LLM,
    K,
)

# LLM
from model.huggingface_model import HuggingfaceModel
from model import get_model

2025-07-10 06:39:55.982557: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-10 06:39:56.005691: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-10 06:39:56.005710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-10 06:39:56.006804: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-10 06:39:56.011396: I tensorflow/core/platform/cpu_feature_guar

AttributeError: jit_init_thread_state(): the LLVM backend is inactive because the LLVM shared library ("libLLVM.so") could not be found! Set the DRJIT_LIBLLVM_PATH environment variable to specify its path.

In [3]:
model = get_model(SLM)
SLM

'meta-llama/Llama-3.2-1B-Instruct'

In [4]:
dataset = load_dataset("Muennighoff/babi", split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [5]:
len(dataset)

18013

In [6]:
import re


def is_correct(prediction: str, answer: str) -> bool:
    prediction = prediction.strip().lower()
    answer = answer.strip().lower()
    return re.search(rf"\b{re.escape(answer)}\b", prediction) is not None

In [9]:
total = 100
correct = 0
dataset = dataset.shuffle().select(range(total))
for i in range(total):
    passage = dataset[i]["passage"]
    question = dataset[i]["question"]
    answer = dataset[i]["answer"]
    input = (
        f"Instruction: Answer with only one word. No explanation."
        f"Context: {passage.strip()}\n"
        f"Question: {question.strip()}\n"
        f"Answer(only one word):"
    )
    output, _ = model.generate(input)
    prediction = output[len(input) :]
    print(input + prediction + "\n")

    if is_correct(prediction, answer):
        correct += 1

print(f"Accuracy: {correct}/{total} = {correct/total:.2%}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: Answer with only one word. No explanation.Context: John and Mary moved to the garden.
Sandra and John moved to the bedroom.
Sandra and John went back to the garden.
Sandra and Daniel journeyed to the hallway.
Question: Where is Daniel?
Answer(only one word): Hallway.

Instruction: Answer with only one word. No explanation.Context: Sheep are afraid of mice.
Mice are afraid of sheep.
Gertrude is a sheep.
Cats are afraid of wolves.
Winona is a sheep.
Emily is a mouse.
Wolves are afraid of cats.
Jessica is a sheep.
Question: What is emily afraid of?
Answer(only one word): sheep.

Instruction: Answer with only one word. No explanation.Context: The box fits inside the chest.
The suitcase is bigger than the container.
The chocolate fits inside the box.
The chest is bigger than the container.
The suitcase is bigger than the chest.
Question: Is the chest bigger than the chocolate?
Answer(only one word): No.

Instruction: Answer with only one word. No explanation.Context: Mary trave

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: Answer with only one word. No explanation.Context: Fred grabbed the football there.
Jeff took the apple there.
Jeff dropped the apple.
Bill picked up the apple there.
Mary travelled to the kitchen.
Mary went back to the hallway.
Bill went to the garden.
Fred travelled to the garden.
Bill passed the apple to Fred.
Fred left the apple.
Fred went back to the hallway.
Fred handed the football to Mary.
Jeff went back to the garden.
Bill picked up the apple there.
Mary travelled to the office.
Mary travelled to the kitchen.
Question: Who gave the football?
Answer(only one word): Fred.

Instruction: Answer with only one word. No explanation.Context: Sandra grabbed the apple.
Sandra put down the apple.
John moved to the kitchen.
Sandra grabbed the apple there.
Mary went back to the garden.
Sandra journeyed to the office.
Mary travelled to the kitchen.
Sandra put down the apple.
John moved to the garden.
Sandra moved to the bathroom.
Sandra went to the office.
John went to the kitc

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Instruction: Answer with only one word. No explanation.Context: Mary journeyed to the bathroom.
Daniel grabbed the milk there.
Daniel dropped the milk there.
Mary went back to the bedroom.
Mary went to the kitchen.
Sandra went to the kitchen.
Daniel picked up the milk there.
Sandra moved to the bathroom.
Question: How many objects is Daniel carrying?
Answer(only one word): Three.

Instruction: Answer with only one word. No explanation.Context: The chocolate fits inside the box of chocolates.
The box is bigger than the box of chocolates.
The chocolate fits inside the container.
The chest is bigger than the box of chocolates.
The box of chocolates fits inside the chest.
Question: Is the chest bigger than the chocolate?
Answer(only one word): Yes.

Instruction: Answer with only one word. No explanation.Context: The bathroom is west of the bedroom.
The bedroom is south of the hallway.
The kitchen is south of the garden.
The garden is east of the office.
The bedroom is north of the garden.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: Answer with only one word. No explanation.Context: Sandra journeyed to the bathroom.
John got the apple there.
John left the apple.
Mary journeyed to the bedroom.
Daniel grabbed the apple.
Daniel went back to the office.
John went back to the garden.
Daniel left the apple.
Daniel travelled to the bedroom.
Mary moved to the hallway.
Sandra moved to the hallway.
Sandra went back to the bathroom.
John went back to the kitchen.
John travelled to the office.
Mary went back to the bedroom.
Daniel went back to the office.
John journeyed to the garden.
Daniel grabbed the apple.
Mary journeyed to the hallway.
Mary went back to the bedroom.
John travelled to the hallway.
Daniel took the football.
Sandra journeyed to the garden.
Sandra went back to the hallway.
John got the milk.
Sandra travelled to the bedroom.
Daniel dropped the football.
Daniel dropped the apple there.
Daniel grabbed the football there.
John journeyed to the garden.
Daniel went to the bedroom.
Mary went back to th

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: Answer with only one word. No explanation.Context: Daniel journeyed to the hallway.
Mary got the football there.
Sandra moved to the hallway.
Sandra moved to the garden.
Question: How many objects is Mary carrying?
Answer(only one word): One.

Instruction: Answer with only one word. No explanation.Context: Bill went to the school.
Fred is either in the kitchen or the park.
Question: Is Bill in the school?
Answer(only one word): Yes.

Instruction: Answer with only one word. No explanation.Context: Greg is a swan.
Lily is a lion.
Bernhard is a lion.
Bernhard is gray.
Julius is a swan.
Julius is yellow.
Brian is a frog.
Brian is yellow.
Greg is yellow.
Question: What color is Lily?
Answer(only one word): Yellow
Note: The

Instruction: Answer with only one word. No explanation.Context: Sheep are afraid of mice.
Winona is a sheep.
Mice are afraid of sheep.
Emily is a sheep.
Wolves are afraid of cats.
Jessica is a sheep.
Gertrude is a mouse.
Cats are afraid of sheep.
Question: W

In [ ]:
model = get_model(LLM)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
model.generate(input)

In [ ]:
total = 100
correct = 0
for i in range(total):
    passage = dataset[i]["passage"]
    question = dataset[i]["question"]
    answer = dataset[i]["answer"]
    input = (
        f"Context: {passage.strip()}\n"
        f"Question: {question.strip()}\n"
        f"Answer(only one word):"
    )
    output = model.generate(input)
    prediction = output[len(input) :].strip()

    if is_correct(prediction, answer):
        correct += 1

print(f"Accuracy: {correct}/{total} = {correct/total:.2%}")